In [3]:
from src.agents import create_agent 

In [4]:
create_agent?

Signature:
create_agent(
    model: 'str | BaseChatModel | SyncOrAsync[[StateT, Runtime[ContextT]], BaseChatModel]',
    tools: 'Sequence[BaseTool | Callable | dict[str, Any]] | ToolNode',
    *,
    middleware: 'Sequence[AgentMiddleware]' = (),
    prompt: 'Prompt | None' = None,
    response_format: 'ToolStrategy[StructuredResponseT] | ProviderStrategy[StructuredResponseT] | type[StructuredResponseT] | None' = None,
    pre_model_hook: 'RunnableLike | None' = None,
    post_model_hook: 'RunnableLike | None' = None,
    state_schema: 'type[StateT] | None' = None,
    context_schema: 'type[ContextT] | None' = None,
    checkpointer: 'Checkpointer | None' = None,
    store: 'BaseStore | None' = None,
    interrupt_before: 'list[str] | None' = None,
    interrupt_after: 'list[str] | None' = None,
    debug: 'bool' = False,
    version: "Literal['v1', 'v2']" = 'v2',
    name: 'str | None' = None,
    **deprecated_kwargs: 'Any',
) -> 'CompiledStateGraph[StateT, ContextT]'
Docstring:
Create

In [12]:
from langchain_openai import ChatOpenAI
from langgraph.store.memory import InMemoryStore
from langchain_core.messages import BaseMessage, HumanMessage

checkpointer = InMemoryStore()

agent = create_agent(
    model =ChatOpenAI(model_name="gpt-4", temperature=0),
    tools = [],
    # checkpointer = checkpointer
)



In [22]:
thread_config = {"configurable": {"thread_id": "deep-research-thread-11thronee1"}}

query = """
Do a deep research on agentech in oklahoma city. you should use the web. Agentech is a new startup in the Insurance Claims space. You need to find out the following:
this is their url: https://www.agentech.com/
1. Who are the founders of the company?
2. What is the funding status of the company?
3. What are the recent news articles about the company?
4. What are the competitors of the company?
5. What are the recent job postings for the company?
"""

initial_input = {"messages": [HumanMessage(content=query)]}

result = agent.invoke(initial_input, config=thread_config)

In [23]:
result

{'messages': [HumanMessage(content='\nDo a deep research on agentech in oklahoma city. you should use the web. Agentech is a new startup in the Insurance Claims space. You need to find out the following:\nthis is their url: https://www.agentech.com/\n1. Who are the founders of the company?\n2. What is the funding status of the company?\n3. What are the recent news articles about the company?\n4. What are the competitors of the company?\n5. What are the recent job postings for the company?\n', additional_kwargs={}, response_metadata={}, id='58e48b3a-ef03-499f-8809-00db73f26112'),
  AIMessage(content='I\'m sorry, but I was unable to find any information related to a startup named "Agentech" in Oklahoma City in the Insurance Claims space. The provided URL (https://www.agentech.com/) does not lead to a valid website. It\'s possible that there may be a typo in the company name or the company may not have a strong online presence yet. If you have any additional information, I\'d be happy to 

In [20]:
# The final output structure might vary, adjust this key if needed.
if result.get('messages') and isinstance(result['messages'], list) and len(result['messages']) > 1:
        final_report_content = result['messages'][-1].content
        # Assuming the report is in the content of the last message
        print("\n--- FINAL REPORT ---")
        print(final_report_content)
else:
        print("Could not find final report in the result.")
        print(result)



--- FINAL REPORT ---
I'm sorry, but I was unable to find any information on a company called Agentech in Oklahoma City. The URL provided does not lead to a valid website. It's possible that there may be a typo in the name or the company is too new to have an online presence. If you have any additional information, I would be happy to continue the research.


In [2]:
from src.agents.agents.middleware.types import AgentState